# Train Network Paper II
In this notebook we lay out the neural network used in the second paper in the series (https://arxiv.org/abs/2102.06230). To create the synthetic spectra used in this work, please run the `Create_Spectra_all.py` code after updating the output region.

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
from keras.backend import clear_session
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Flatten, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
from pickle import dump
from keras.backend import clear_session
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
import statistics
from scipy import interpolate
from scipy.stats import gaussian_kde

In [ ]:
#------------------------ INPUTS ----------------------------#
# Set Input Parameters
sim_names = ['10k-red-broad/BOND']#, '10k-red/PNe_2020', '10k-red/SNR_2008']
ref_dir = '/home/carterrhea/Dropbox/CFHT/Analysis-Paper2/SyntheticData/'
syn_num = 30000  # Number of Synthetic Data per simulation
ensemble_num = 10  # Number of layers in ensemble

In [ ]:
print("# -- Read in Reference Spectra -- #")
# IF RESOLUTION IS SAMPLED USE THIS
# This gets us the wavenumbers we are going to interpolate on
# We use this spectra because it is a nice looking (can easily see the lines -- not relevant here)
# spectra at a resolution of 5000 exactly. We want to keep that sampling :)
ref_SN3 = fits.open(ref_dir+'Reference-Spectrum-SN3.fits')[1].data
channel = []
counts = []
for chan in ref_SN3:
    channel.append(chan[0])
    counts.append(chan[1])
wavenumbers_syn_SN3 = channel


ref_SN2 = fits.open(ref_dir+'Reference-Spectrum-SN2.fits')[1].data
channel = []
counts = []
for chan in ref_SN2:
    channel.append(chan[0])
    counts.append(chan[1])
wavenumbers_syn_SN2 = channel


ref_SN1 = fits.open(ref_dir+'Reference-Spectrum-SN1.fits')[1].data
channel = []
counts = []
for chan in ref_SN1:
    channel.append(chan[0])
    counts.append(chan[1])
wavenumbers_syn_SN1 = channel


In [ ]:

print("# -- Read in SN3 -- #")
# Read in Fits SN3
Counts_SN3 = []  # List containing the spectra
Param_dict_SN3 = {}
sim_ct = 0
for sim_name in sim_names:
    print('We are on simutlation: %s'%sim_name)
    output_dir = ref_dir+sim_name+'/'
    plot_dir = ref_dir+sim_name+'/'
    for spec_ct in range(syn_num):
        if spec_ct%1000 == 0:
            print('  We are on spectrum number %i'%spec_ct)
        spectrum = fits.open(output_dir+'Spectrum_SN3_%i.fits'%spec_ct)
        header = spectrum[0].header
        spec = spectrum[1].data
        channel = []
        counts = []
        for chan in spec:
            channel.append(chan[0])
            counts.append(chan[1])
        # interpolate
        f = interpolate.interp1d(channel, counts, kind='slinear')
        # Get fluxes of interest
        coun = f(wavenumbers_syn_SN3[2:-2])  # Need to cut the size just a little bit of the interpolating region otherwise we wont be able to interpolate!
        Counts_SN3.append(np.array(coun))
        Param_dict_SN3[sim_ct*syn_num+spec_ct] = [header['Halpha'], header['NII6548'], header['NII6583'], header['SII6716'], header['SII6731']]
    sim_ct += 1
print("# -- Read in SN2 -- #")
# Read in Fits SN2
Counts_SN2 = []  # List containing the spectra
Param_dict_SN2 = {}
sim_ct = 0
for sim_name in sim_names:
    print('We are on simutlation: %s'%sim_name)
    output_dir = ref_dir+sim_name+'/'
    plot_dir = ref_dir+sim_name+'/'
    for spec_ct in range(syn_num):
        if spec_ct%1000 == 0:
            print('We are on spectrum number %i'%spec_ct)
        spectrum = fits.open(output_dir+'Spectrum_SN2_%i.fits'%spec_ct)
        header = spectrum[0].header
        spec = spectrum[1].data
        channel = []
        counts = []
        for chan in spec:
            channel.append(chan[0])
            counts.append(chan[1])
        # interpolate
        f = interpolate.interp1d(channel, counts, kind='slinear')
        # Get fluxes of interest
        coun = f(wavenumbers_syn_SN2[2:-2])
        Counts_SN2.append(np.array(coun))
        Param_dict_SN2[sim_ct*syn_num+spec_ct] = [header['OIII5007'],header['OIII4959'], header['Hbeta']]
    sim_ct += 1
print("# -- Read in SN1 -- #")
# Read in Fits SN1
Counts_SN1 = []  # List containing the spectra
Param_dict_SN1 = {}
sim_ct = 0
for sim_name in sim_names:
    print('We are on simutlation: %s'%sim_name)
    output_dir = ref_dir+sim_name+'/'
    plot_dir = ref_dir+sim_name+'/'
    for spec_ct in range(syn_num):
        if spec_ct%1000 == 0:
            print('We are on spectrum number %i'%spec_ct)
        spectrum = fits.open(output_dir+'Spectrum_SN1_%i.fits'%spec_ct)
        header = spectrum[0].header
        spec = spectrum[1].data
        channel = []
        counts = []
        for chan in spec:
            channel.append(chan[0])
            counts.append(chan[1])
        # interpolate
        f = interpolate.interp1d(channel, counts, kind='slinear')
        # Get fluxes of interest
        coun = f(wavenumbers_syn_SN1[2:-2])
        Counts_SN1.append(np.array(coun))
        Param_dict_SN1[sim_ct*syn_num+spec_ct] = [header['OII3726'], header['OII3729']]
    sim_ct += 1

In [ ]:
# Combine the Filters for each observation
Counts = [np.concatenate((sn1,sn2,sn3), axis=0) for sn1,sn2,sn3 in zip(Counts_SN1,Counts_SN2,Counts_SN3)]
# Define values of interest
s2_s1 = [np.arcsinh(np.round(val[4]/val[3], 2)) for val in list(Param_dict_SN3.values())]  # s2/s1
s1s2_ha = [np.arcsinh(np.round((val[3]+val[4])/val[0], 2)) for val in list(Param_dict_SN3.values())]   # (s1+s2)/ha
n2_ha = [np.arcsinh(np.round(val[2]/val[0], 2)) for val in list(Param_dict_SN3.values())]  # n2/ha
n2_s1s2 = [np.arcsinh(np.round(val[2]/(val[3]+val[4]), 2)) for val in list(Param_dict_SN3.values())]  # n2/(s1+s2)
o3_hb = [np.arcsinh(np.round(val[0]/val[2], 2)) for val in list(Param_dict_SN2.values())]  # O3/hb
o2_hb = [np.arcsinh(np.round((O2[0]+O2[1])/hb[2], 2)) for O2,hb in list(zip(Param_dict_SN1.values(),Param_dict_SN2.values()))]  # O2/hb
o2_o3 = [np.arcsinh(np.round((O2[0]+O2[1])/O3[0], 2)) for O2,O3 in list(zip(Param_dict_SN1.values(),Param_dict_SN2.values()))]  # O2/O3
ha_hb = [np.arcsinh(np.round(ha[0]/hb[2], 2)) for ha,hb in list(zip(Param_dict_SN3.values(),Param_dict_SN2.values()))]  # ha/hb
n1_n2 = [np.arcsinh(np.round(val[1]/val[2], 2)) for val in list(Param_dict_SN3.values())]

## Define the Model

In [ ]:
# ------------------------- DEFINE MODEL -------------------------- #

activation = 'relu'  # activation function
initializer = 'normal'  # model initializer
batch_size = 16  # number of data fed into model at once
max_epochs = 25  # maximum number of interations
early_stopping_min_delta = 0.0001
early_stopping_patience = 4
reduce_lr_factor = 0.5
reuce_lr_epsilon = 0.009
reduce_lr_patience = 2
reduce_lr_min = 0.00008
loss_function = 'Huber'
metrics_ = ['accuracy', 'mae', 'mse']



clear_session()
def create_model():
    model = Sequential([
        Dense(units=1024, kernel_initializer=initializer, activation=activation,
              kernel_regularizer=l2(0.00005)),
        Dropout(0.25),
        Dense(units=1024, kernel_initializer=initializer, activation=activation,
              kernel_regularizer=l2(0.00005)),
        Dropout(0.15),
        Dense(units=512, kernel_initializer=initializer, activation=activation,
              kernel_regularizer=l2(0.00005)),
        Dense(9),
    ])
    return model

optimizer = Adam(learning_rate=0.0004)

early_stopping = EarlyStopping(monitor='val_loss', min_delta=early_stopping_min_delta,
                                       patience=early_stopping_patience, verbose=2, mode='min')

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, epsilon=reuce_lr_epsilon,
                                  patience=reduce_lr_patience, min_lr=reduce_lr_min, mode='min', verbose=2)



## Define Data sets

In [ ]:
# ----------------------------- TRAIN and VALIDATE ------------------------------ #
print("# -- Train Network -- #")
# Get number of sucessful cloudy runs
syn_num_pass = len(Counts)
train_div = int(0.7*syn_num_pass)  # Percent of synthetic data to use as training
valid_div = int(0.9*syn_num_pass)  # Percent of synthetic data used as training and validation (validation being the difference between this and train_div)
test_div = int(1.0*syn_num_pass)
# Set training set
TraningSet = np.array(Counts[:train_div])
Traning_init_labels = np.array((s2_s1[0:train_div], s1s2_ha[0:train_div], n2_ha[0:train_div], n2_s1s2[0:train_div], o3_hb[0:train_div], o2_hb[0:train_div], o2_o3[0:train_div], ha_hb[0:train_div], n1_n2[0:train_div])).T
# Set validation set
ValidSet = np.array(Counts[train_div:valid_div])
Valid_init_labels = np.array((s2_s1[train_div:valid_div], s1s2_ha[train_div:valid_div], n2_ha[train_div:valid_div], n2_s1s2[train_div:valid_div], o3_hb[train_div:valid_div], o2_hb[train_div:valid_div], o2_o3[train_div:valid_div], ha_hb[train_div:valid_div], n1_n2[train_div:valid_div])).T


## Train the model using deep ensembling

In [ ]:
for step in range(ensemble_num):
    model = create_model()
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics_)
    model.fit(TraningSet, Traning_init_labels, validation_data=(ValidSet, Valid_init_labels),
          epochs=max_epochs,batch_size= batch_size, verbose=2, callbacks=[reduce_lr,early_stopping])
    model.save_weights('./ensembling_weights/step_%i'%step)


model.save('Sitelle-Line-Ratios-Model')

## Test the model -- this is important for later if you want to implement this network in your own code!

In [ ]:
print("# -- Test Network -- #")
TestSet = np.array(Counts[valid_div:test_div])
TestSetLabels = np.array((s2_s1[valid_div:test_div], s1s2_ha[valid_div:test_div], n2_ha[valid_div:test_div], n2_s1s2[valid_div:test_div], o3_hb[valid_div:test_div], o2_hb[valid_div:test_div], o2_o3[valid_div:test_div], ha_hb[valid_div:test_div], n1_n2[valid_div:test_div])).T
# Apply Deep ensembling predictions
ensembling_predictions = {}  # {ensemble number: list of predictions}
for step in range(ensemble_num):
    # Restore the weights
    model = create_model()
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics_)
    model.load_weights('./ensembling_weights/step_%i'%step)
    predictions = model.predict(TestSet)  # If not testing use predictions=model(TestSet) as it is faster
    ensembling_predictions[step] = predictions

In [ ]:
predictions = np.zeros((len(TestSet),9))
prediction_prob = np.zeros((len(TestSet),9))  # (test_i, [ratios])
prediction_std = np.zeros((len(TestSet),9))

for test in range(len(TestSet)):  # Get prediction for each test
    ens_pred_final = np.zeros((9,ensemble_num))
    for ens in range(ensemble_num):  # get prediction of each network
        ens_pred = ensembling_predictions[ens]  # Get ensemble number predictions
        ens_test_pred = ens_pred[test]  # Get test prediction for given model in ensemble
        ens_pred_final[:,ens] = ens_test_pred  # Sum up the predictions
    prediction_prob[test] = ens_pred_final.mean(axis=1)
    prediction_std[test] = ens_pred_final.std(axis=1)

## Error calculations, plotting, and pickle dumps

In [ ]:
# ------------------------------ ERROR CALCULATIONS ------------------#
print("# -- Error Calculations -- #")
errs_s2_s1 = [100*(prediction_prob[i,0] - TestSetLabels[i,0])/TestSetLabels[i,0] for i in range(len(prediction_prob))]
errs_s1s2_ha = [100*(prediction_prob[i,1] - TestSetLabels[i,1])/TestSetLabels[i,1] for i in range(len(prediction_prob))]
errs_n2_ha = [100*(prediction_prob[i,2] - TestSetLabels[i,2])/TestSetLabels[i,2] for i in range(len(prediction_prob))]
errs_n1_s1s2 = [100*(prediction_prob[i,3] - TestSetLabels[i,3])/TestSetLabels[i,3] for i in range(len(prediction_prob))]
errs_o3_hb = [100*(prediction_prob[i,4] - TestSetLabels[i,4])/TestSetLabels[i,4] for i in range(len(prediction_prob))]
errs_o2_hb = [100*(prediction_prob[i,5] - TestSetLabels[i,5])/TestSetLabels[i,5] for i in range(len(prediction_prob))]
errs_o2_o3 = [100*(prediction_prob[i,6] - TestSetLabels[i,6])/TestSetLabels[i,6] for i in range(len(prediction_prob))]
errs_ha_hb = [100*(prediction_prob[i,7] - TestSetLabels[i,7])/TestSetLabels[i,7] for i in range(len(prediction_prob))]
errs_n1_n2 = [100*(prediction_prob[i,8] - TestSetLabels[i,8])/TestSetLabels[i,8] for i in range(len(prediction_prob))]

In [ ]:

fig, axs = plt.subplots(2, 4, figsize=(20,8))
sns.distplot(errs_s2_s1, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[0,0])
#axs[0,0].set_xlim(-5,5)
axs[0,0].set_title(r'S[II]$\lambda$6731/S[II]$\lambda$6716')# -- std:%.2f'%(np.std(errs_s2_s1)))
axs[0,0].set_ylabel('Density', fontweight='bold', fontsize=14)


sns.distplot(errs_n2_ha, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[0,1])
#axs[0,1].set_xlim(-50,50)
axs[0,1].set_title(r'N[II]$\lambda$6584/H$\alpha(6563)\AA$')#' -- std:%.2f'%(np.std(errs_n2_ha)))

sns.distplot(errs_s1s2_ha, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[1,0])
#axs[1,0].set_xlim(-50,50)
axs[1,0].set_title(r'(S[II]$\lambda6717$+S[II]$\lambda6731$)/H$\alpha(6563)\AA$')# -- std:%.2f'%(np.std(errs_s1s2_ha)))
axs[1,0].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)
axs[1,0].set_ylabel('Density', fontweight='bold', fontsize=14)

sns.distplot(errs_n1_s1s2, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[1,1])
#axs[1,1].set_xlim(-50,50)
axs[1,1].set_title(r'N[II]$\lambda$6584/(S[II]$\lambda6717$+S[II]$\lambda6731$)')#' -- std:%.2f'%(np.std(errs_n1_s1s2)))
axs[1,1].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)

sns.distplot(errs_o2_hb, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[0,2])
#axs[1,1].set_xlim(-50,50)
axs[0,2].set_title(r'O[II]$\lambda$3726/H$\beta$')#' -- std:%.2f'%(np.std(errs_n1_s1s2)))
#axs[0,2].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)

sns.distplot(errs_o3_hb, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[1,2])
#axs[1,1].set_xlim(-50,50)
axs[1,2].set_title(r'OIII$\lambda$5007/H$\beta$')#' -- std:%.2f'%(np.std(errs_n1_s1s2)))
axs[1,2].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)

sns.distplot(errs_o2_o3, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[0,3])
#axs[1,1].set_xlim(-50,50)
axs[0,3].set_title(r'OII$\lambda$3726/OIII$\lambda$5007')#' -- std:%.2f'%(np.std(errs_n1_s1s2)))
#axs[0,3].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)

sns.distplot(errs_ha_hb, hist=False, kde=True,  kde_kws={"shade": True},
             bins=20, color = 'darkmagenta',
             ax=axs[1,3])
#axs[1,1].set_xlim(-50,50)
axs[1,3].set_title(r'H$\alpha$/H$\beta$')#' -- std:%.2f'%(np.std(errs_n1_s1s2)))
axs[1,3].set_xlabel('Relative Error (%)', fontweight='bold', fontsize=14)



plt.savefig(plot_dir+'Ratio-Errors.pdf', type='pdf')

error_txt = open(plot_dir+'err_std.txt', 'w+')
errs = [errs_s2_s1, errs_n2_ha, errs_s1s2_ha, errs_n1_s1s2, errs_o2_o3, errs_o3_hb, errs_o2_hb, errs_ha_hb, errs_n1_n2]
errs_name = ['errs_s2_s1', 'errs_n2_ha', 'errs_s1s2_ha', 'errs_n1_s1s2', 'errs_o2_o3', 'errs_o3_hb', 'errs_o2_hb', 'errs_ha_hb', 'errs_n1_n2']
for err,err_name in zip(errs,errs_name):
    std_err = np.std(err)
    mean_ = np.median(err)
    print("The standard deviation for %s is %.2f"%(str(err_name), std_err))
    print("The mean for %s is %.2f"%(str(err_name), mean_))
    error_txt.write("The standard deviation for %s is %.2f\n"%(str(err_name), std_err))
    error_txt.write("The mean for %s is %.2f\n"%(str(err_name), mean_))
error_txt.close()

In [ ]:
pickle.dump(errs_s2_s1, open('errs_s2_s1.pkl', 'wb'))
pickle.dump(errs_n2_ha, open('errs_n2_ha.pkl', 'wb'))
pickle.dump(errs_s1s2_ha, open('errs_s1s2_ha.pkl', 'wb'))
pickle.dump(errs_n1_s1s2, open('errs_n1_s1s2.pkl', 'wb'))
pickle.dump(errs_o2_o3, open('errs_o2_o3.pkl', 'wb'))
pickle.dump(errs_o3_hb, open('errs_o3_hb.pkl', 'wb'))
pickle.dump(errs_o2_hb, open('errs_o2_hb.pkl', 'wb'))
pickle.dump(errs_ha_hb, open('errs_ha_hb.pkl', 'wb'))
pickle.dump(errs_n1_n2, open('errs_n1_n2.pkl', 'wb'))